In [8]:
import numpy as np
import pandas as pd
import re, nltk, spacy, gensim
import os
# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

import warnings
warnings.filterwarnings("ignore")

from nltk.corpus import stopwords

In [9]:
folder_path = ['comments1k/0_9.txt', 'comments1k/1_7.txt']

# create an empty list to store the data
data = []

# loop through all files in the folder
for filename in folder_path:
        # read the file
        with open(filename, "r") as f:
            content = f.read()
        
        # append the data to the list
        data.append({"data": content})

# create a DataFrame from the data
df = pd.DataFrame(data)

# display the DataFrame
df.head()


,data
0,Bromwell High is a cartoon comedy. It ran at t...
1,"If you like adult comedy cartoons, like South ..."


In [10]:
df['data'] = df['data'].str.replace('&\w+;','')
df['data'] = df['data'].apply(lambda x: re.sub('<.*?>', '', x))

df['data'] = df['data'].str.lower()

df['data'] = df['data'].str.replace('[^\w\s]',' ')
# Load the stop words from NLTK
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

df['data'] = df['data'].apply(lambda x:' '.join([w for w in x.split() if w not in stop_words]))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\saima_x4lzx52\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
df.iloc[0]

data    bromwell high cartoon comedy ran time programs...
Name: 0, dtype: object

In [12]:
pprint(df.iloc[1])

data    like adult comedy cartoons like south park nea...
Name: 1, dtype: object


In [13]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data_words_0 = list(sent_to_words(df.iloc[0]))
data_words_1 = list(sent_to_words(df.iloc[1]))
print(data_words_0[:1])
print()
print(data_words_1[:1])



[['bromwell', 'high', 'cartoon', 'comedy', 'ran', 'time', 'programs', 'school', 'life', 'teachers', 'years', 'teaching', 'profession', 'lead', 'believe', 'bromwell', 'high', 'satire', 'much', 'closer', 'reality', 'teachers', 'scramble', 'survive', 'financially', 'insightful', 'students', 'see', 'right', 'pathetic', 'teachers', 'pomp', 'pettiness', 'whole', 'situation', 'remind', 'schools', 'knew', 'students', 'saw', 'episode', 'student', 'repeatedly', 'tried', 'burn', 'school', 'immediately', 'recalled', 'high', 'classic', 'line', 'inspector', 'sack', 'one', 'teachers', 'student', 'welcome', 'bromwell', 'high', 'expect', 'many', 'adults', 'age', 'think', 'bromwell', 'high', 'far', 'fetched', 'pity']]

[['like', 'adult', 'comedy', 'cartoons', 'like', 'south', 'park', 'nearly', 'similar', 'format', 'small', 'adventures', 'three', 'teenage', 'girls', 'bromwell', 'high', 'keisha', 'natella', 'latrina', 'given', 'exploding', 'sweets', 'behaved', 'like', 'bitches', 'think', 'keisha', 'good',

In [14]:
# Create the LDA model
lda = gensim.ldamodel.LDA(data_words_0, num_topics=10)

# Get the top 8 words for each topic
topics = lda.top_topics(8)

# Print the top 8 words for each topic
for topic, words in topics:
    print(topic, ': '.join(words))

AttributeError: module 'gensim' has no attribute 'ldamodel'

In [ ]:
%pip install lda

  Using cached lda-2.0.0.tar.gz (320 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached pbr-3.1.1-py2.py3-none-any.whl (99 kB)
  Running setup.py install for lda: started
  Running setup.py install for lda: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  DEPRECATION: lda is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  error: subprocess-exited-with-error
  
  × Running setup.py install for lda did not run successfully.
  │ exit code: 1
  ╰─> [61 lines of output]
      c:\Users\saima_x4lzx52\AppData\Local\Programs\Python\Python311\Lib\site-packages\setuptools\dist.py:771: UserWarning: Usage of dash-separated 'description-file' will not be supported in future versions. Please use the underscore name 'description_file' instead
        warnings.warn(
      c:\Users\saima_x4lzx52\AppData\Local\Programs\Python\Python311\Lib\site-packages\setuptools\dist.py:771: UserWarning: Usage of dash-separated 'author-email' will not be supported in future versions. Please use the

In [ ]:
model = lda.LDA(n_topics=20, n_iter=1500, random_state=1)
model.fit(data_words_0) # model.fit_transform(X) is also available
topic_word = model.topic_word_ # model.components_ also works
n_top_words = 8
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    print('Topic {}: {}'.format(i, ''.join(topic_words)))
doc_topic = model.doc_topic_
for i in range(10):
    print("{} (top topic: {})".format(titles[i], doc_topic[i].argmax()))

NameError: name 'lda' is not defined